##### 三类股票分类，
##### 使用未来三天的平均change来作为标签值y，并用0.33, 0.67分位值作为三类标签的划分边界

##### 数据: 基本指标 
##### 模型: SVR 和  random select

In [1]:
# 引入头文件
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score
import numpy as np
from sklearn.model_selection import GridSearchCV

In [2]:
import tensorflow as tf
import pandas as pd
from pandas import DataFrame
import numpy as np
from numpy import row_stack,column_stack
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY,YEARLY
# from mpl_finance import quotes_historical_yahoo_ohlc, candlestick_ohlc
from matplotlib.pylab import date2num
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation,Dropout
import tushare as ts
import datetime
import time

d:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
ts.set_token('29ea8fa19e717f1a3d3618796f87067ee07219783a9e01422eb90ef8')
pro = ts.pro_api()

# df = ts.get_hist_data('601318',start='2009-01-01',end='2018-01-12') #一次性获取全部日k线数据
df = pro.daily(ts_code='601318.SH', start_date = '20110101', end_date = '20181118')
df = df.iloc[::-1]
df.head()

df.to_csv('Pingan-from-20110101-to-20181118.csv', index=False)

df = pd.read_csv('Pingan-from-20110101-to-20181118.csv')
df.head()

input_data = df[['open','high','low','pre_close','change','pct_chg','vol','amount','close']]


In [4]:
# input_data['change'].quantile([0.33,0.5, 0.67,])
quantile033 = input_data['change'].quantile([0.33]).iloc[0]
quantile067 = input_data['change'].quantile([0.67]).iloc[0]

In [5]:
change = input_data['change']
ave_change_column = [ ]
for i in range(0, len(change)):
    if i >= len(change) - 3:
        ave_change = change[i]
        
    else:
        ave_change = (change[i+1] + change[i+2] + change[i+3]) / 3
    
    ave_change_column.append(ave_change)

In [6]:
input_data['ave_change'] = ave_change_column

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
input_data.head(15)

,open,high,low,pre_close,change,pct_chg,vol,amount,close,ave_change
0,56.85,57.60,56.50,56.16,0.75,1.34,245626.82,1400499.338,56.91,-1.300000
1,56.59,56.80,54.80,56.91,-2.05,-3.60,427554.25,2368305.774,54.86,-0.943333
2,54.94,54.94,51.45,54.86,-2.27,-4.14,947078.65,4971851.955,52.59,-0.156667
3,52.97,54.25,51.80,52.59,0.42,0.80,659013.87,3494756.547,53.01,-0.006667
4,52.90,53.90,51.90,53.01,-0.98,-1.85,282919.36,1494174.281,52.03,0.316667
5,51.84,52.55,51.70,52.03,0.09,0.17,258047.33,1343674.168,52.12,-0.046667
6,52.27,53.20,52.00,52.12,0.87,1.67,225559.26,1189009.625,52.99,-1.046667
7,53.22,53.44,52.55,52.99,-0.01,-0.02,195120.49,1034239.321,52.98,-0.993333
8,52.71,52.92,51.80,52.98,-1.00,-1.89,154439.85,806532.104,51.98,-0.433333
9,51.75,51.97,49.80,51.98,-2.13,-4.10,308565.65,1561578.586,49.85,-0.356667


In [8]:
input_x = input_data.iloc[:, :9]
input_y = input_data.iloc[:, 9]
input_x.head()

,open,high,low,pre_close,change,pct_chg,vol,amount,close
0,56.85,57.60,56.50,56.16,0.75,1.34,245626.82,1400499.338,56.91
1,56.59,56.80,54.80,56.91,-2.05,-3.60,427554.25,2368305.774,54.86
2,54.94,54.94,51.45,54.86,-2.27,-4.14,947078.65,4971851.955,52.59
3,52.97,54.25,51.80,52.59,0.42,0.80,659013.87,3494756.547,53.01
4,52.90,53.90,51.90,53.01,-0.98,-1.85,282919.36,1494174.281,52.03


In [9]:
input_y.head()

0   -1.300000
1   -0.943333
2   -0.156667
3   -0.006667
4    0.316667
Name: ave_change, dtype: float64

In [10]:
## 补0
input_x.fillna(0 , inplace=True)
# output_x.fillna(0, inplace=True)

In [11]:

# 2 分割训练数据和测试数据
# 随机采样25%作为测试 75%作为训练
x_train, x_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.2, shuffle=False)

# from sklearn.model_selection import train_test_split
# train_test_split(X, y, test_size=0.2, shuffle=False)


In [12]:
# x_test.head()

In [13]:
# y_test[0:5]

In [14]:
save_test_data = x_test.copy()
save_test_data['ave_change'] = y_test

In [15]:

# 3 训练数据和测试数据进行标准化处理
ss_x = StandardScaler()
x_train = ss_x.fit_transform(x_train)
x_test = ss_x.transform(x_test)

ss_y = StandardScaler()
y_train = ss_y.fit_transform(y_train.values.reshape(-1, 1))
y_test = ss_y.transform(y_test.values.reshape(-1, 1))

In [16]:
# # 归一化
# ss_x = StandardScaler()
# input_x = ss_x.fit_transform(input_x)
# output_x = ss_x.fit_transform(output_x)
# ss_y = StandardScaler()
# input_y = ss_y.fit_transform(input_y.values.reshape(-1, 1))

In [17]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVR(kernel="rbf",), param_grid={"C":[0.1, 1, 10], "gamma": [1, 0.1, 0.01]}, cv=4)
grid.fit(x_train, y_train)
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best parameters are {'C': 0.1, 'gamma': 0.1} with a score of -0.00


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [67]:
###  TODO 调参
# 4.1 支持向量机模型进行学习和预测
# 线性核函数配置支持向量机
# linear_svr = SVR(kernel="linear")
# 训练
# 自动选择合适的参数
# svr = GridSearchCV(SVR(), param_grid={"kernel": ("linear", 'rbf'), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)})
svr = SVR(kernel="rbf",C = 0.7, gamma = 1 )
# svr.fit(x, y)
svr.fit(x_train, y_train)
# 预测 保存预测结果
svr_y_predict = svr.predict(x_test)

# 多项式核函数配置支持向量机 *** 可以配置参数
poly_svr = SVR(kernel="rbf",gamma = 0.1)
# 训练
poly_svr.fit(x_train, y_train)
# 预测 保存预测结果
poly_svr_y_predict = poly_svr.predict(x_test)

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
real_svr_y_predict = ss_y.inverse_transform(svr_y_predict) 
real_svr_y_predict

array([-0.17151828,  0.0207984 ,  0.00366343,  0.04340546, -0.0085338 ,
       -0.01078955,  0.03542578,  0.03574891,  0.10399143, -0.15154713,
        0.4465442 , -0.08144426, -0.07331847, -0.00753646, -0.03444357,
       -0.06787806,  0.167507  , -0.18071999, -0.08689955,  0.2137406 ,
        0.11561988,  0.05017115,  0.21698937, -0.09058896,  0.06835884,
        0.27750986,  0.15406702, -0.12821875,  0.34668411,  0.43352012,
        0.26744771, -0.11392559, -0.05233062,  0.18208389, -0.34390032,
        0.27029893,  0.22282086,  0.23569578,  0.43251012,  0.34648682,
        0.39128923, -0.07806478,  0.1883364 ,  0.27999159,  0.21121585,
        0.06223206,  0.48466952,  0.48949447,  0.13951253,  0.19695549,
        0.24478325,  0.15626591,  0.18398999,  0.10306169,  0.66036537,
        0.0840425 ,  0.01247396, -0.11854508,  0.07653389,  0.31741509,
       -0.04733089,  0.08857342,  0.0877528 , -0.00472785,  0.10533288,
        0.24320582,  0.00506174, -0.00222643,  0.06331323, -0.08

In [69]:
y_test = ss_y.inverse_transform(y_test) 

In [70]:
y_test[0:10]

array([[ 0.04701752],
       [ 0.0120412 ],
       [-0.04370231],
       [ 0.00439013],
       [ 0.04592451],
       [ 0.30933993],
       [ 0.24375933],
       [ 0.35087431],
       [ 0.1432024 ],
       [ 0.17162066]])

In [71]:
len(real_svr_y_predict), len(y_test)

(382, 382)

In [72]:
### TODO 待完成   crossvalidation调整参数

#(517, 517)


In [73]:
# 5 模型评估
# 线性核函数 模型评估
print("线性核函数支持向量机的默认评估值为：", svr.score(x_test, y_test))
print("线性核函数支持向量机的R_squared值为：", r2_score(y_test, svr_y_predict))
print("线性核函数支持向量机的均方误差为:", mean_squared_error(ss_y.inverse_transform(y_test),
                                              ss_y.inverse_transform(svr_y_predict)))
print("线性核函数支持向量机的平均绝对误差为:", mean_absolute_error(ss_y.inverse_transform(y_test),
                                                 ss_y.inverse_transform(svr_y_predict)))
# 对多项式核函数模型评估
# print("对多项式核函数的默认评估值为：", poly_svr.score(x_test, y_test))
# print("对多项式核函数的R_squared值为：", r2_score(y_test, poly_svr_y_predict))
# print("对多项式核函数的均方误差为:", mean_squared_error(ss_y.inverse_transform(y_test),
#                                            ss_y.inverse_transform(poly_svr_y_predict)))
# print("对多项式核函数的平均绝对误差为:", mean_absolute_error(ss_y.inverse_transform(y_test),
#                                               ss_y.inverse_transform(poly_svr_y_predict)))

线性核函数支持向量机的默认评估值为： -3.070180173419221
线性核函数支持向量机的R_squared值为： -3.070180173419221
线性核函数支持向量机的均方误差为: 0.08185950358904608
线性核函数支持向量机的平均绝对误差为: 0.2216566424223383


In [74]:

test_y = y_test
predict_y = real_svr_y_predict

In [75]:
trend_list = [1,]
for i in range(1, len(predict_y)):
    if predict_y[i] <= quantile033:
        trend_list.append(-1)
    elif quantile033 <= predict_y[i] <= quantile067:
        trend_list.append(0)
    else:
        trend_list.append(1)

real_trend_list = [1,]
for i in range(1, len(test_y)):
    if test_y[i] <= quantile033:
        real_trend_list.append(-1)
    elif quantile033 <= test_y[i] <= quantile067:
        real_trend_list.append(0)
    else:
        real_trend_list.append(1)
        
#     if ave_change < quantile033:
#         trend_column.append(-1)
#     elif quantile033 <= ave_change <= quantile067:
#         trend_column.append(0)
#     else:
#         trend_column.append(1)




print(len(test_y), len(predict_y))

pair_list = []
true_count = 0
false_count = 0
for i in range(len(predict_y)-1):
    pair_list.append([real_trend_list[i], trend_list[i]])
    if real_trend_list[i] == trend_list[i]:
        true_count = true_count + 1
    else:
        false_count = false_count + 1

print(pair_list)
print(true_count, false_count)


382 382
[[1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [1, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [1, 0], [1, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 0], [1, 1], [1, 0], [0, 0], [0, 1], [-1, 1], [0, 1], [0, 0], [0, 0], [1, 0], [0, -1], [0, 1], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1], [0, 0], [0, 0], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [0, 0], [0, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [1, 0], [1, 0], [0, 0], [-1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [-1, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 1], [0, 0], [1, 0], [1, 0], [1, 0], [0, 0], [0, -1], [0, -1], [0, 0], [0, 0], [0, -1], [0, 0], [0, -1], [0, -1], [0, -1], [0, -1], [0, 0], [0, -1], [0, -1], [0, 0], [0, -1], [0, -1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, -1], [0, 0], [0, -1], [0, 0], [0, -1], [0, -1], [1, 0], [1, 0], [0, -1], [0, 0], [0, -1], [1, 0], [0, -1], [0, 0], [0, 0], [0, 0], [0, -1], [1, 0], 

In [76]:
accuracy = true_count/ len(predict_y)
accuracy

0.5130890052356021

In [77]:
real_df = pd.DataFrame(real_trend_list)
pred_df = pd.DataFrame(trend_list)
acc_df = pd.concat((real_df[0],pred_df[0]), axis=1)

temp_list = []
temp_df =  real_df[0] - pred_df[0]
for i in range(0, len(temp_df)):
    temp_list.append([i, temp_df[i]])
    
%matplotlib qt
plt.clf()
plt.scatter( *zip(*temp_list), )
# plt.scatter(real_df.index.values, pred_df[0],color='blue', marker='^')
# plt.scatter(real_df.index.values, real_df[0],color='red', marker='.')
plt.title('Trend Prediction Accuracy of SVR = {}'.format(accuracy))
plt.xlabel('Days')
plt.ylabel('Trend Prediction')
plt.show()

In [29]:
len(temp_df)

382

In [30]:
temp_df.value_counts()

 1    117
 0    116
-1     95
-2     38
 2     16
Name: 0, dtype: int64

In [31]:
%matplotlib qt
plt.clf()
plt.plot(list(range(len(predict_y))), predict_y, color='b',label = 'predict ave change')
plt.plot(list(range(len(test_y))), test_y,  color='r',label = 'true ave change')
plt.title('SVR change price prediction')
plt.xlabel('Days')
plt.ylabel('change price')
plt.legend()
plt.show()

In [32]:
#计算标准差
def mserror(y, y_pred):
    i=0
    mse = 0
    for i in range (len(y_pred)):
        mse = mse + np.mean((y[i] - y_pred[i]) ** 2)
        i += 1
        return mse

mse = mserror( test_y, predict_y)
print(mse)

0.05360071549827622


In [33]:
list3 = [list(a) for a in zip(predict_y, test_y)]

In [34]:
list3[0:10]

[[-0.1715182832915712, array([0.06])],
 [0.020798398226070682, array([-0.04666667])],
 [0.003663431085193852, array([-0.21666667])],
 [0.04340545698078285, array([-0.07])],
 [-0.008533802135328367, array([0.05666667])],
 [-0.01078954524587333, array([0.86])],
 [0.03542577587183956, array([0.66])],
 [0.035748907674143224, array([0.98666667])],
 [0.10399142626601934, array([0.35333333])],
 [-0.1515471292011722, array([0.44])]]

In [35]:
from collections import Counter

result = Counter(trend_list)
print ("预测数据", result)
result = Counter(real_trend_list)
print ("真实数据", result)

预测数据 Counter({0: 259, 1: 92, -1: 31})
真实数据 Counter({1: 158, -1: 119, 0: 105})


以下是随机对比模型

In [36]:
import random
random_list = []
for i in range(len(test_y)):
    random_list.append(random.randint(-1,1))

len(random_list)

382

In [78]:
pair_list = []
true_count = 0
false_count = 0
for i in range(len(predict_y)-1):
    pair_list.append([real_trend_list[i], random_list[i]])
    if real_trend_list[i] == random_list[i]:
        true_count = true_count + 1
    else:
        false_count = false_count + 1

In [79]:
accuracy = true_count/ len(predict_y)
accuracy

0.32460732984293195

In [50]:
real_df = pd.DataFrame(real_trend_list)
pred_df = pd.DataFrame(random_list)
acc_df = pd.concat((real_df[0],pred_df[0]), axis=1)

temp_list = []
temp_df =  real_df[0] - pred_df[0]
for i in range(0, len(temp_df)):
    temp_list.append([i, temp_df[i]])
    
%matplotlib qt
plt.clf()
plt.scatter( *zip(*temp_list), )
# plt.scatter(real_df.index.values, pred_df[0],color='blue', marker='^')
# plt.scatter(real_df.index.values, real_df[0],color='red', marker='.')
plt.title('Trend Prediction Accuracy of Random Select = {}'.format(accuracy))
plt.xlabel('Days')
plt.ylabel('Trend Prediction')
plt.show()

In [52]:
len(temp_df)

382

In [51]:
temp_df.value_counts()

 0    143
-1     79
 1     70
 2     59
-2     31
Name: 0, dtype: int64

In [49]:
%matplotlib qt
plt.clf()
plt.plot(list(range(len(predict_y))), predict_y, color='b',label = 'predict ave change')
plt.plot(list(range(len(test_y))), test_y,  color='r',label = 'true ave change')
plt.title('SVR change price prediction')
plt.xlabel('Days')
plt.ylabel('change price')
plt.legend()
plt.show()

In [41]:
new_data = save_test_data

In [42]:
new_data.head()

,open,high,low,pre_close,change,pct_chg,vol,amount,close,ave_change
1527,37.49,38.26,37.44,37.51,0.31,0.83,1250054.58,4732657.328,37.82,0.060000
1528,37.90,38.07,37.61,37.82,0.14,0.37,678663.87,2568647.819,37.96,-0.046667
1529,37.80,38.38,37.80,37.96,-0.05,-0.13,619325.39,2359210.184,37.91,-0.216667
1530,37.95,38.28,37.82,37.91,0.09,0.24,505383.86,1920918.772,38.00,-0.070000
1531,37.95,38.10,37.73,38.00,-0.18,-0.47,548573.55,2080732.893,37.82,0.056667


In [43]:
new_data.shape , len(real_svr_y_predict)

((382, 10), 382)

In [44]:
new_data['pred_change'] = real_svr_y_predict

In [45]:
new_data.tail()

,open,high,low,pre_close,change,pct_chg,vol,amount,close,ave_change,pred_change
1904,64.40,65.14,63.75,64.62,0.04,0.0619,458511.64,2956910.272,64.66,0.146667,0.207267
1905,63.78,65.57,63.50,64.66,0.35,0.5413,520997.95,3362080.743,65.01,0.080000,0.185607
1906,65.05,65.09,63.60,65.01,-1.06,-1.6305,573814.07,3680152.361,63.95,-1.060000,0.323508
1907,64.00,65.14,63.77,63.95,1.15,1.7983,500579.86,3231462.676,65.10,1.150000,0.095015
1908,65.00,66.39,64.69,65.10,0.15,0.2304,619879.08,4063786.057,65.25,0.150000,0.284266


In [46]:
len(real_trend_list), len(trend_list) , len(random_list)

(382, 382, 382)

In [47]:
new_data['real_trend'] = real_trend_list
new_data['pred_trend'] = trend_list
new_data['random_trend'] = random_list

In [48]:
new_data.to_csv('Result-SVR&Random-Pingan-from-20080101-to-20181118.csv', index=False)